This notebook demonstrates how to use the photometric signal-to-noise routines defined in `sims_photUtils/../photUtils/SignalToNoise.py`.

First, let's just do some setup.  We will create a list of 20 random spectra drawn from the library of Kurucz spectra stored in `sims_sed_library`.  We will normalize them randomly in the u band.

In [ ]:
import os
import numpy as np
from lsst.utils import getPackageDir
from lsst.sims.photUtils import Bandpass, Sed

nSpectra = 20

sedDir = os.path.join(getPackageDir('sims_sed_library'), 'starSED', 'kurucz')
bandpassDir = os.path.join(getPackageDir('throughputs'), 'baseline')

np.random.seed(42)

# create a random list of SED file names drawn from sedDir
sedFullNameList = np.array(os.listdir(sedDir))
sedDexes = np.unique(np.random.random_integers(0, len(sedFullNameList), nSpectra))
sedNameList = sedFullNameList[sedDexes]

# create a random list of normalizing magnitudes
magNormList = np.random.random_sample(nSpectra)*10.0 + 18.0

# read in the r bandpass
rBandpass = Bandpass()
rBandpass.readThroughput(os.path.join(bandpassDir, 'total_r.dat'))

sedList = []
for name, norm in zip(sedNameList, magNormList):

    fullName = os.path.join(sedDir, name)

    # instantiate and read in the SED
    spectrum = Sed()
    spectrum.readSED_flambda(fullName)

    # normalize the SED to have the specified magnitude in the r band
    ff = spectrum.calcFluxNorm(norm, rBandpass)
    spectrum.multiplyFluxNorm(ff)
    sedList.append(spectrum)

We normalized our example spectra in the u band.  Let's say we want to calculate signal to noise quantities in the i band.  We can do that, by loading the file `total_i.dat` from the `baseline/` sub-directory of the `throughputs` eups pacakge.

In [ ]:
bandpassDir = os.path.join(getPackageDir('throughputs'), 'baseline')

total_i_bandpass = Bandpass()
total_i_bandpass.readThroughput(os.path.join(bandpassDir, 'total_i.dat'))

`total_i.dat` contains the combined throughputof the lenses, the mirrors, the i filter, and the standard LSST atmosphere (airmass of 1.2).  We could have constructed that throughput by hand by loading the throughputs for each of these components separately and then combining them with teh `readThroughputList` method.  This is useful if we want to consider throughputs a different airmasses (the `throughputs` package contains a few atmospheric throughput files computed by MODTRAN at different airmasses).

The cell below construct `total_i_bandpass` 'by hand' and verifies that it is the same (within 10 significant figures) as `total_i.dat`.

In [ ]:
test_i_bandpass = Bandpass()
componentList = ['detector.dat', 'm1.dat', 'm2.dat', 'm3.dat', 
                 'lens1.dat', 'lens2.dat', 'lens3.dat', 'filter_i.dat', 'atmos.dat']
test_i_bandpass.readThroughputList(componentList, rootDir=bandpassDir)

np.testing.assert_array_equal(test_i_bandpass.wavelen, total_i_bandpass.wavelen)
np.testing.assert_allclose(test_i_bandpass.sb,
                           total_i_bandpass.sb,
                           rtol=10)

For some of the calculations below, we will require the throughput of just the hardware (so: everything except the atmosphere).  This is another use for the `readThroughputList` method.

In [ ]:
hardware_i_bandpass = Bandpass()
componentList = ['detector.dat', 'm1.dat', 'm2.dat', 'm3.dat', 
                 'lens1.dat', 'lens2.dat', 'lens3.dat', 'filter_i.dat']
hardware_i_bandpass.readThroughputList(componentList, rootDir=bandpassDir)

We are also going to require a simulated sky emissivity spectrum.  The `baseline/` sub-directory of `throughputs` also contains the file `darksky.dat`, which is our fiducial sky emissivity spectrum.

In [ ]:
skySed = Sed()
skySed.readSED_flambda(os.path.join(bandpassDir, 'darksky.dat'))

Finally, there are a bunch of photometric parametrs (readnoise, gain, etc.) that will be required for signal-to-noise calculations.  Those are contained (and passed around) in the class `PhotometricParameters`.

In [ ]:
from lsst.sims.photUtils import PhotometricParameters

photParams = PhotometricParameters()

You can specify parameter values by hand when you call the `PhotometricParameters` constructor.  If you do nothing, you will just get LSST default values.

In [ ]:
help(PhotometricParameters.__init__)

We are also going to be required to specify the `FWHMeff` (the full-width at half maximum in arcseconds of an idealized Gaussian PSF; this characterizes the seeing of an observation).  If you don't want to figure out that value on your own, the class `LSSTdefaults` contains assumed LSST default values (as well as values for other observing parameters like m5).

In [ ]:
from lsst.sims.photUtils import LSSTdefaults

defaults = LSSTdefaults()

print help(defaults.FWHMeff)
print '\n'
print help(defaults.effwavelen)
print '\n'
print help(defaults.m5)
print '\n'
print help(defaults.gamma)

There are two ways to calculate the signal to noise of a source.

The methods `calcSNR_sed` and `calcMagError_sed` assume that you have an SED for your source, and SED for sky emissivity, the throughput for the bandpass plus atmosphere, the throughput for the hardware only, as well as teh photometric parameters described above.  These methods then calculate counts expected form the source and compare them with counts expected from the sky and other hardware-related noise contributors and give you a signal to noise ratio.

The methods `calcSNR_m5` and `calcMagErro_m5` assume you do not have a full sky emissivity SED, but you do have an m5 value for the observation as well as magnitudes for your source, a total hardware plus atmosphere throughput curve, and the photometric parameters described above.

Below, we demonstrate that these two methods give comparabel results.

First we must calculate the m5 value in the i bandpass for our observation.  We can use the method `calcM5`.

In [ ]:
import lsst.sims.photUtils.SignalToNoise as snr
m5 = snr.calcM5(skySed, total_i_bandpass, hardware_i_bandpass, photParams, FWHMeff=0.8)
print m5

Calculate the signal-to-noise ration using the SED.

In [ ]:
snr_sed_list = []
for spectrum in sedList:
    val = snr.calcSNR_sed(spectrum, total_i_bandpass, skySed, hardware_i_bandpass,
                         photParams, FWHMeff=0.8)
    snr_sed_list.append(val)

snr_sed_list = np.array(snr_sed_list)

Calculate the signal-to-noise ratio using only magnitudes.  Note that, in addition to the signal-to-noise, `calcSNR_m5` also returns the `gamma` parameter from arXiv:0805.233 equation (5).  This can be passed back into `calcSNR_m5` to avoid re-calculation if necessary.

In [ ]:
magList = []
for spectrum in sedList:
    mm = spectrum.calcMag(total_i_bandpass)
    magList.append(mm)
magList = np.array(magList)

snr_m5_list, gamma = snr.calcSNR_m5(magList, total_i_bandpass, m5, photParams)

Compare.

In [ ]:
print 'maximum deviation in snr: ',np.abs(snr_sed_list - snr_m5_list).max()

print 'value of snr at maximum deviation: ', \
       snr_sed_list[np.argmax(np.abs(snr_sed_list - snr_m5_list))]

Calculate the magnitude error using the SED.

In [ ]:
mag_error_sed_list = []
for spectrum in sedList:
    sigma = snr.calcMagError_sed(spectrum, total_i_bandpass, skySed,
                                hardware_i_bandpass, photParams,
                                FWHMeff=0.8)
    mag_error_sed_list.append(sigma)

mag_error_sed_list = np.array(mag_error_sed_list)

Calculate the magnitude error using magnitudes.  Note that, in addition to the magnitude error, `calcMagError_m5` also returns the `gamma` parameter from arXiv:0805.233 equation (5).  This can be passed back into `calcSNR_m5` to avoid re-calculation if necessary.

In [ ]:
mag_error_m5_list, gamma = snr.calcMagError_m5(magList, total_i_bandpass, m5, photParams)

Compare.

In [ ]:
print 'maximum deviation in magnitude error: ',np.abs(mag_error_sed_list - mag_error_m5_list).max()

print 'value of magnitude at maximum deviation: ', \
       magList[np.argmax(np.abs(mag_error_sed_list - mag_error_m5_list))]